In [1]:
# Importing Dependencies
from splinter import Browser
from bs4 import BeautifulSoup

In [2]:
# Path to chromedriver
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
# Setting the executable path
path = {'executable_path': '/usr/local/bin/chromedriver'}
# Initializing chrome browser in splinter
browser = Browser('chrome', **path)

In [ ]:
################################################################################
# Collecting latest headline & paragraph text from NASA's 'Mars News' Website
################################################################################

In [4]:
# Visiting URL
NASA_url = 'https://mars.nasa.gov/news/'
browser.visit(NASA_url)

True

In [5]:
# Converting browser html to soup 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

slide_element = soup.select_one('ul.item_list li.slide')

In [6]:
# Finding content title
slide_element.find("div", class_='content_title')

<div class="content_title"><a href="/news/8436/why-this-martian-full-moon-looks-like-candy/" target="_self">Why This Martian Full Moon Looks Like Candy</a></div>

In [7]:
# Using parent element to find first tag
title = slide_element.find("div", class_='content_title').get_text()
title

'Why This Martian Full Moon Looks Like Candy'

In [8]:
# Using parent element to find paragraph text
p = slide_element.find('div', class_="article_teaser_body").get_text()
p

"For the first time, NASA's Mars Odyssey orbiter has caught the Martian moon Phobos during a full moon phase. Each color in this new image represents a temperature range detected by Odyssey's infrared camera."

In [ ]:
###########################################
# Aquiring the Featured JPL Space Image
###########################################

In [9]:
# Visiting URL with splinter
JPL_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(JPL_url)

In [10]:
# Finding and clicking 'full image' button
full_image_button = browser.find_by_id('full_image')
full_image_button.click()

In [11]:
# Finding 'more info' button and clicking that
browser.is_element_present_by_text('more info', wait_time=1)
MoreInfo_button = browser.find_link_by_partial_text('more info')
MoreInfo_button.click()

In [12]:
# Parsing html with soup
result_html = browser.html
featured_img_soup = BeautifulSoup(result_html, 'html.parser')

In [13]:
# Finding the relative image url
featured_img_url_rel = featured_img_soup.select_one('figure.lede a img').get("src")
featured_img_url_rel

'/spaceimages/images/largesize/PIA14925_hires.jpg'

In [14]:
# Using base url + relative url to create absolute url
featured_img_url = f'https://www.jpl.nasa.gov{featured_img_url_rel}'
featured_img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14925_hires.jpg'

In [ ]:
###############################################
# Scraping the Latest Mars Weather Tweet
###############################################

In [15]:
Twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(Twitter_url)

In [16]:
html = browser.html
mars_weather_soup = BeautifulSoup(html, 'html.parser')

In [17]:
# First, find a tweet with the data-name `Mars Weather`
mars_weather_tweet = mars_weather_soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})

In [18]:
# Next, search within the tweet for the p tag containing the tweet text
mars_weather = mars_weather_tweet.find('p', 'tweet-text').get_text()
mars_weather

'InSight sol 162 (2019-05-12) low -100.2ºC (-148.3ºF) high -20.3ºC (-4.5ºF)\nwinds from the SW at 4.5 m/s (10.1 mph) gusting to 14.3 m/s (32.0 mph)\npressure at 7.50 hPapic.twitter.com/23uEPf5baF'

In [ ]:
##########################################################################
# Obtaining Images for Mars Hemispheres from USGS Astrogeology Website
##########################################################################

In [19]:
USGS_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(USGS_url)

In [20]:
hemisphere_image_urls = []

# Collecting a list of all hemisphere html links
hemisphere_links = browser.find_by_css("a.product-item h3")

# Looping through the hemisphere links, clicking each link, finding the sample anchor, and returning the href
for i in range(len(hemisphere_links)):
    hemisphere = {}
    
    # Finding the elements on each loop
    browser.find_by_css("a.product-item h3")[i].click()
    
    # Finding the 'sample image' anchor tag and extracting the href
    sample_tag = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_tag['href']
    
    # Collecting Hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Appending hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    # Navigating backwards
    browser.back()
    

In [21]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [ ]:
#####################################################################################################
# Visiting the Mars Facts Webpage and Using Pandas to Scrape the Table Containing Facts About Mars
#####################################################################################################

In [22]:
import pandas as pd
df = pd.read_html('https://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [23]:
# Converting Pandas Dataframe to html
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>

In [24]:
browser.quit()